---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

<!-- After digesting the instructions, you can delete this cell, these are assignment instructions and do not need to be included in your final submission.  -->

{{< include instructions.qmd >}} 

# Code 

Provide the source code used for this section of the project here.

If you're using a package for code organization, you can import it at this point. However, make sure that the **actual workflow steps**—including data processing, analysis, and other key tasks—are conducted and clearly demonstrated on this page. The goal is to show the technical flow of your project, highlighting how the code is executed to achieve your results.

If relevant, link to additional documentation or external references that explain any complex components. This section should give readers a clear view of how the project is implemented from a technical perspective.

Remember, this page is a technical narrative, NOT just a notebook with a collection of code cells, include in-line Prose, to describe what is going on.

## Clean the News Data

In [2]:
# import required libraries
import os
import json
import re
import pandas as pd

In [23]:
# function for cleaning the text

def string_cleaner(input_string):
    try:
        out = re.sub(r"""
                    [,.;@#?!&$-]+
                    \ *          
                    """,
                    " ",
                    input_string, flags=re.VERBOSE)
        out = re.sub('[’.]+', '', out)
        out = re.sub(r'\\u[0-9a-fA-F]{4}', '', out)
        out = re.sub(r'\s+', ' ', out)
        out = out.lower()
    except:
        print("ERROR")
        out = ''
    return out

In [24]:
# Function to clean news data
def clean_news_data(raw_data_dir, clean_data_dir):
    
    # Iterate through raw data files
    for file_name in os.listdir(raw_data_dir):
        if file_name.endswith("_raw_text.json"):  # Process only raw data files
            
            # Load the raw data
            raw_file_path = os.path.join(raw_data_dir, file_name)
            with open(raw_file_path, 'r') as raw_file:
                raw_data = json.load(raw_file)
            
            # Clean the data
            clean_data = {}
            for article in raw_data:
                title = article.get('title', '')
                description = article.get('description', '')
                
                if title and description:
                    clean_title = string_cleaner(title)
                    clean_description = string_cleaner(description)
                    clean_data[clean_title] = clean_description
            
            # Save the cleaned data to a new file
            clean_file_name = file_name.replace("_raw_text.json", "_clean_news.json")
            clean_file_path = os.path.join(clean_data_dir, clean_file_name)
            with open(clean_file_path, 'w') as clean_file:
                json.dump(clean_data, clean_file, indent=4)

In [25]:
# Define directories

# Directory with raw data files
raw_data_dir = "../../data/raw-data/News_Drivers"  

# Directory for cleaned data files
clean_data_dir = "../../data/processed-data/News_drivers"  


In [26]:
# Clean the news data
clean_news_data(raw_data_dir, clean_data_dir)

## Clean the Drivers Standings

In [32]:
input_file = "../../data/raw-data/Driver_standings/driver_standings_2000_2023.json"
output_file = "../../data/processed-data/driver_standings_2000_2023.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

In [33]:
with open(input_file, 'r') as f:
    data = json.load(f)

# Prepare a list to store extracted records
cleaned_data = []

# Loop through each season in the JSON
for season, season_data in data.items():
    standings_lists = season_data.get('MRData', {}).get('StandingsTable', {}).get('StandingsLists', [])
    
    for standings in standings_lists:
        driver_standings = standings.get('DriverStandings', [])
        
        for entry in driver_standings:
            # Extract required fields
            position = entry.get('position', '')
            points = entry.get('points', '')
            wins = entry.get('wins', '')
            driver = entry.get('Driver', {})
            constructors = entry.get('Constructors', [])
            
            # Extract driver and constructor details
            given_name = driver.get('givenName', '')
            family_name = driver.get('familyName', '')
            constructor_id = constructors[0].get('constructorId', '') if constructors else ''
            constructor_name = constructors[0].get('name', '') if constructors else ''
            
            # Append the record to the cleaned data list
            cleaned_data.append({
                "Season": season,
                "Position": position,
                "FirstName": given_name,
                "LastName": family_name,
                "Constructor_ID": constructor_id,
                "Constructor_Name": constructor_name,
                "Points": points,
                "Wins": wins
            })

# Convert the list to a Pandas DataFrame
df = pd.DataFrame(cleaned_data)

# Save the DataFrame to a CSV file
df.to_csv(output_file, index=False)

## Clean the Circuit Information

In [46]:
input_file = "../../data/raw-data/circuit_data.json"
output_file = "../../data/processed-data/circuit_data_clean.csv"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

In [47]:
# Ensure the output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Read the JSON file
with open(input_file, 'r') as f:
    data = json.load(f)

# Extract circuit data
circuits = data.get('MRData', {}).get('CircuitTable', {}).get('Circuits', [])

# Prepare a list to store extracted records
cleaned_data = []

for circuit in circuits:
    circuit_id = circuit.get('circuitId', '')
    circuit_name = circuit.get('circuitName', '')
    country = circuit.get('Location', {}).get('country', '')
    latitude = circuit.get('Location', {}).get('lat', '')
    longitude = circuit.get('Location', {}).get('long', '')
    
    # Append to the list
    cleaned_data.append({
        "Circuit_ID": circuit_id,
        "Circuit_Name": circuit_name,
        "Country": country,
        "Latitude": latitude,
        "Longitude": longitude
    })

# Convert the list to a Pandas DataFrame
df = pd.DataFrame(cleaned_data)

# Save the DataFrame to a CSV file
df.to_csv(output_file, index=False)

## Clean the Race data

In [7]:
## Cleaning all the race_data and appending them in to a single csv file 

# input output directory
input_dir = "../../data/raw-data/"
# output directory
output_file = "../../data/processed-data/all_race_results_cleaned.csv"

# creating an output file
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# initialize a list to hold all results
all_combined_results = []

# process each JSON file in the input directory
# they are the only .json files in the directory
for file_name in os.listdir(input_dir):
    # process only JSON files
    if file_name.endswith(".json"): 
        file_path = os.path.join(input_dir, file_name)
        #print(f"Processing file: {file_path}")
        
        # read the JSON file
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # extract races from the JSON
        races = data.get('MRData', {}).get('RaceTable', {}).get('Races', [])
        
        # prepare a list to hold flattened race results for this file
        file_results = []

        # loop through each race and flatten its data
        for race in races:
            # extract required information 
            race_info = { 
                "season": race.get("season", ""),
                "round": race.get("round", ""),
                "raceName": race.get("raceName", ""),
                "url": race.get("url",""),
                "circuitName": race.get("Circuit", {}).get("circuitName", ""),
                "locality": race.get("Circuit", {}).get("Location", {}).get("locality", ""),
                "country": race.get("Circuit", {}).get("Location", {}).get("country", ""),
                "lat": race.get("Circuit", {}).get("Location", {}).get("lat", ""),
                "long": race.get("Circuit", {}).get("Location", {}).get("long", ""),
                "date": race.get("date", ""),
            }
            
            # extract results and combine with useful details
            results = race.get("Results", [])
            for result in results:
                # combine race-level and result-level data
                combined_data = {**race_info, **result}
                # add flattened driver and constructor details
                combined_data.update({
                    "driverId": result.get("Driver", {}).get("driverId", ""),
                    "driverGivenName": result.get("Driver", {}).get("givenName", ""),
                    "driverFamilyName": result.get("Driver", {}).get("familyName", ""),
                    "constructorId": result.get("Constructor", {}).get("constructorId", ""),
                    "constructorName": result.get("Constructor", {}).get("name", ""),
                    "status": result.get("status", ""),
                    "timeMillis": result.get("Time", {}).get("millis", ""),
                    "time": result.get("Time", {}).get("time", "")
                })
                file_results.append(combined_data)

        # append the results for this file to the combined list
        all_combined_results.extend(file_results)

# aonvert the combined results to a Pandas DataFrame
df = pd.DataFrame(all_combined_results)

# aave the combined DataFrame to a CSV file
df.to_csv(output_file, index=False)



In [4]:
df = pd.read_csv("../../data/processed-data/all_race_results_cleaned.csv")

In [5]:
df.columns

Index(['season', 'round', 'raceName', 'url', 'circuitName', 'locality',
       'country', 'lat', 'long', 'date', 'number', 'position', 'positionText',
       'points', 'Driver', 'Constructor', 'grid', 'laps', 'status', 'Time',
       'FastestLap', 'driverId', 'driverGivenName', 'driverFamilyName',
       'constructorId', 'constructorName', 'timeMillis', 'time'],
      dtype='object')

In [6]:
df = df.drop(columns=['positionText', 'number', 'FastestLap', 'Driver', 'Constructor', 'Time'], axis=1)

In [7]:
df.head()

,season,round,raceName,url,circuitName,locality,country,lat,long,date,...,grid,laps,status,driverId,driverGivenName,driverFamilyName,constructorId,constructorName,timeMillis,time
0,2010,1,Bahrain Grand Prix,http://en.wikipedia.org/wiki/2010_Bahrain_Gran...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,2010-03-14,...,3,49,Finished,alonso,Fernando,Alonso,ferrari,Ferrari,5960396.0,1:39:20.396
1,2010,1,Bahrain Grand Prix,http://en.wikipedia.org/wiki/2010_Bahrain_Gran...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,2010-03-14,...,2,49,Finished,massa,Felipe,Massa,ferrari,Ferrari,5976495.0,+16.099
2,2010,1,Bahrain Grand Prix,http://en.wikipedia.org/wiki/2010_Bahrain_Gran...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,2010-03-14,...,4,49,Finished,hamilton,Lewis,Hamilton,mclaren,McLaren,5983578.0,+23.182
3,2010,1,Bahrain Grand Prix,http://en.wikipedia.org/wiki/2010_Bahrain_Gran...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,2010-03-14,...,1,49,Finished,vettel,Sebastian,Vettel,red_bull,Red Bull,5999195.0,+38.799
4,2010,1,Bahrain Grand Prix,http://en.wikipedia.org/wiki/2010_Bahrain_Gran...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,2010-03-14,...,5,49,Finished,rosberg,Nico,Rosberg,mercedes,Mercedes,6000609.0,+40.213


In [8]:
# check for null values
df.isnull().sum()

season                 0
round                  0
raceName               0
url                    0
circuitName            0
locality               0
country                0
lat                    0
long                   0
date                   0
position               0
points                 0
grid                   0
laps                   0
status                 0
driverId               0
driverGivenName        0
driverFamilyName       0
constructorId          0
constructorName        0
timeMillis          1291
time                1291
dtype: int64

Interpretation:
- The missing vlaues in the 'TimeMillis' and 'time' columns are for those records where the driver did not finish the race. 

In [9]:
# drop timeMillis and time columns
df = df.drop(columns=['time', 'timeMillis'], axis=1)

In [10]:
df.isnull().sum()

season              0
round               0
raceName            0
url                 0
circuitName         0
locality            0
country             0
lat                 0
long                0
date                0
position            0
points              0
grid                0
laps                0
status              0
driverId            0
driverGivenName     0
driverFamilyName    0
constructorId       0
constructorName     0
dtype: int64

## Cleaning Weather Data

In [47]:
weather_df = pd.read_csv("../../data/raw-data/weather/race_data_with_weather.csv")

In [48]:
weather_df.isnull().sum()

season      0
raceName    0
url         0
weather     0
dtype: int64

In [49]:
weather_df['weather']

0                                                  Sunny
1                      Overcast with light rain at start
2                                     Mainly cloudy, dry
3                                           Cloudy, rain
4                                     Mainly cloudy, dry
                             ...                        
117    Sunny with temperatures reaching up to 27 °C (...
118    Dry start, with heavy rain and thunderstorm/mo...
119                                                 Rain
120                                                Sunny
121                                          Warm, Sunny
Name: weather, Length: 122, dtype: object

#### we will try to categorise the weather description into one of the following categories:
1. Sunny
2. Cloudy
3. Rainy
4. Windy

In [53]:
def classify_weather(weather_description):

    weather_description = weather_description.lower()
    
    if "sunny" in weather_description or "fine" in weather_description or "clear" in weather_description or "dry" in weather_description:
        return "Sunny"
    elif "cloudy" in weather_description or "overcast" in weather_description or "cloud" in weather_description:
        return "Cloudy"
    elif "rain" in weather_description or "thunderstorms" in weather_description or "drizzle" in weather_description:
        return "Rainy"
    elif "windy" in weather_description:
        return "Windy"
    # If no match, classify as "Not Available"
    else:
        return "Not Available" 

In [54]:
# call classify_weather()
weather_df['weather_class'] = weather_df['weather'].apply(classify_weather)

# Save the updated DataFrame to a CSV file
output_csv = "../../data/processed-data/classified_weather_data.csv"
weather_df.to_csv(output_csv, index=False)

In [55]:
weather_df['weather_class'].value_counts()

weather_class
Sunny            95
Cloudy           24
Rainy             2
Not Available     1
Name: count, dtype: int64